# Repetition Code Syndrome Extraction

This notebook demonstrates stabilizer simulation of syndrome extraction for the **repetition code** using the `paulimer` library.

The repetition code encodes a single logical qubit in $n$ physical data qubits. Its stabilizers are $Z_i Z_{i+1}$ for adjacent qubit pairs. Measuring these stabilizers (syndrome extraction) detects bit-flip errors without collapsing the logical state.

## Setup

In [1]:
from paulimer import OutcomeCompleteSimulation
from paulimer import UnitaryOpcode
from paulimer import SparsePauli

## Quantum Primitives

Helper functions for Bell pair preparation and Pauli measurements.

In [2]:
def h(sim, target):
    sim.apply_unitary(unitary_op= UnitaryOpcode.Hadamard, support=[target])

def zz(qubit_a, qubit_b):
    return SparsePauli.z(qubit_a) * SparsePauli.z(qubit_b)

def measure_zz(sim, qubit_a, qubit_b):
    bit = sim.measure(zz(qubit_a, qubit_b))
    return bit

## Syndrome Extraction Circuit

Extracts all $n-1$ stabilizer syndromes by directly measuring each $Z_i Z_{i+1}$ observable.

In [3]:
def repetition_code_syndrome_extraction(sim, data_qubits):
    num_stabilizers = len(data_qubits) - 1
    syndrome = []
    for i in range(num_stabilizers):
        bit = measure_zz(sim, data_qubits[i], data_qubits[i + 1])
        syndrome.append(bit)
    return syndrome

## Verification

To verify correctness, we apply Hadamards to all data qubits *before* syndrome extraction, creating |+⟩ states. After extraction, `is_stabilizer` checks that each $Z_i Z_{i+1}$ is indeed a stabilizer with the measured syndrome sign. This confirms the circuit correctly projects onto the stabilizer eigenspace.

In [ ]:
def test_syndrome_extraction(sim, data_qubit_count: int):
    num_stabilizers = data_qubit_count - 1
    data_qubits = list(range(data_qubit_count))
    
    for q in data_qubits:
        h(sim, q)
    
    syndrome = repetition_code_syndrome_extraction(sim, data_qubits)

    for i in range(num_stabilizers):
        sim.is_stabilizer(zz(data_qubits[i], data_qubits[i + 1]), sign_parity=[syndrome[i]])

## Run Tests

Test syndrome extraction for repetition codes of sizes 3 to 199. Each test creates a fresh simulation, runs the verification, and implicitly asserts correctness via `is_stabilizer`.

In [5]:
for n in range(3, 200):
    sim = OutcomeCompleteSimulation()
    test_syndrome_extraction(sim, n)